In [40]:
import pandas as pd
import chardet
import numpy as np

IBGE files has a different encoding a different separator. So we're making a function to load them

In [41]:
def load_path_from_ibge_auto_encoding(path):
    # First we need to detect the encoding
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
        print(f"Will load {path} with encoding {encoding}")
    return pd.read_csv(path, encoding=encoding, sep=';')

We ran these functions and we know that the encoding is `Windows-1252`

From the dictionary:
```
Pessoas	Demografia	V01031	0 a 4 anos
Pessoas	Demografia	V01032	5 a 9 anos
Pessoas	Demografia	V01033	10 a 14 anos
Pessoas	Demografia	V01034	15 a 19 anos
Pessoas	Demografia	V01035	20 a 24 anos
Pessoas	Demografia	V01036	25 a 29 anos
Pessoas	Demografia	V01037	30 a 39 anos
Pessoas	Demografia	V01038	40 a 49 anos
Pessoas	Demografia	V01039	50 a 59 anos
Pessoas	Demografia	V01040	60 a 69 anos
Pessoas	Demografia	V01041	70 anos ou mais
```

```
Pessoas	Alfabetização	V00644	15 a 19 anos
Pessoas	Alfabetização	V00645	20 a 24 anos
Pessoas	Alfabetização	V00646	25 a 29 anos
Pessoas	Alfabetização	V00647	30 a 34 anos
Pessoas	Alfabetização	V00648	35 a 39 anos
Pessoas	Alfabetização	V00649	40 a 44 anos
Pessoas	Alfabetização	V00650	45 a 49 anos
Pessoas	Alfabetização	V00651	50 a 54 anos
Pessoas	Alfabetização	V00652	55 a 59 anos
Pessoas	Alfabetização	V00653	60 a 64 anos
Pessoas	Alfabetização	V00654	65 a 69 anos
Pessoas	Alfabetização	V00655	70 a 79 anos
Pessoas	Alfabetização	V00656	80 anos ou mais

Pessoas	Alfabetização	V00748	Pessoas alfabetizadas, 15 a 19 anos
Pessoas	Alfabetização	V00749	Pessoas alfabetizadas, 20 a 24 anos
Pessoas	Alfabetização	V00750	Pessoas alfabetizadas, 25 a 29 anos
Pessoas	Alfabetização	V00751	Pessoas alfabetizadas, 30 a 34 anos
Pessoas	Alfabetização	V00752	Pessoas alfabetizadas, 35 a 39 anos
Pessoas	Alfabetização	V00753	Pessoas alfabetizadas, 40 a 44 anos
Pessoas	Alfabetização	V00754	Pessoas alfabetizadas, 45 a 49 anos
Pessoas	Alfabetização	V00755	Pessoas alfabetizadas, 50 a 54 anos
Pessoas	Alfabetização	V00756	Pessoas alfabetizadas, 55 a 59 anos
Pessoas	Alfabetização	V00757	Pessoas alfabetizadas, 60 a 64 anos
Pessoas	Alfabetização	V00758	Pessoas alfabetizadas, 65 a 69 anos
Pessoas	Alfabetização	V00759	Pessoas alfabetizadas, 70 a 79 anos
Pessoas	Alfabetização	V00760	Pessoas alfabetizadas, 80 anos ou mais
```

In [44]:
def get_education_df(path, id_col, name_col):
    df = pd.read_csv(path, encoding='Windows-1252', sep=';', low_memory=False)
    
    # List of columns to convert from string to float
    columns_to_convert = [
        'V00748', 'V00644', 'V00749', 'V00645', 'V00750', 'V00646', 
        'V00751', 'V00647', 'V00752', 'V00648', 'V00753', 'V00649',
        'V00754', 'V00650', 'V00755', 'V00651', 'V00756', 'V00652',
        'V00757', 'V00653', 'V00758', 'V00654', 'V00759', 'V00655', 
        'V00760', 'V00656'
    ]
    
    # We're going to clean all columns except the first two
    exclude_columns = [id_col, name_col]
    columns_to_clean = df.columns.difference(exclude_columns)
    
    # Replace '.' with NaN only in the specified columns
    df[columns_to_clean] = df[columns_to_clean].replace('.', np.nan)
    
    # Convert the specified columns to numeric, coercing errors to NaN
    df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')
    
    # Drop rows where any of the specified columns have NaN values
    df = df.dropna(subset=columns_to_clean)
    
    # Convert the specified columns to float type
    df[columns_to_convert] = df[columns_to_convert].astype(float)
    
    # Doing this weird replacement to ensure that if we're dividing by zero, that is, total pop is 0, then we just fill it with nan and replace the whole thing with 0
    df['perc alfa 15 a 19'] =  df['V00748'].divide(df['V00644'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 20 a 24'] =  df['V00749'].divide(df['V00645'].replace(0, np.nan)).fillna(0)   
    df['perc alfa 25 a 29'] =  df['V00750'].divide(df['V00646'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 30 a 34'] =  df['V00751'].divide(df['V00647'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 35 a 39'] =  df['V00752'].divide(df['V00648'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 40 a 44'] =  df['V00753'].divide(df['V00649'].replace(0, np.nan)).fillna(0) 
    df['perc alfa 45 a 49'] =  df['V00754'].divide(df['V00650'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 50 a 54'] =  df['V00755'].divide(df['V00651'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 55 a 59'] =  df['V00756'].divide(df['V00652'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 60 a 64'] =  df['V00757'].divide(df['V00653'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 65 a 69'] =  df['V00758'].divide(df['V00654'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 70 a 79'] =  df['V00759'].divide(df['V00655'].replace(0, np.nan)).fillna(0)  
    df['perc alfa 80+']     =  df['V00760'].divide(df['V00656'].replace(0, np.nan)).fillna(0)
    df['perc alfa mean'] = df[['perc alfa 15 a 19', 'perc alfa 20 a 24', 'perc alfa 25 a 29', 
                                                 'perc alfa 30 a 34', 'perc alfa 35 a 39', 'perc alfa 40 a 44', 
                                                 'perc alfa 45 a 49', 'perc alfa 50 a 54', 'perc alfa 55 a 59', 
                                                 'perc alfa 60 a 64', 'perc alfa 65 a 69', 'perc alfa 70 a 79', 
                                                 'perc alfa 80+']].mean(axis=1)
    df['total pop'] = df['V00644'] + df['V00645'] + df['V00646'] + df['V00647'] + df['V00648'] + df['V00649'] + df['V00650'] + df['V00651'] + df['V00652'] + df['V00653'] + df['V00654'] + df['V00655'] + df['V00656']
    
    result = df[[id_col, name_col, 'perc alfa mean', 'total pop',
                         'perc alfa 15 a 19', 'perc alfa 20 a 24', 'perc alfa 25 a 29', 
                         'perc alfa 30 a 34', 'perc alfa 35 a 39', 'perc alfa 40 a 44', 
                         'perc alfa 45 a 49', 'perc alfa 50 a 54', 'perc alfa 55 a 59', 
                         'perc alfa 60 a 64', 'perc alfa 65 a 69', 'perc alfa 70 a 79', 
                         'perc alfa 80+', 
                         'V00748', 'V00749','V00750','V00751','V00752','V00753','V00754',
                         'V00755','V00756','V00757','V00758','V00759','V00760','V00644',
                         'V00645','V00646','V00647','V00648','V00649','V00650','V00651',
                         'V00652','V00653','V00654','V00655','V00656']]
    result_sorted = result.sort_values(by='perc alfa mean', ascending=False)
    return result_sorted
        
district_df = get_education_df("../data/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/Agregados_por_Distrito_csv/Agregados_por_distritos_alfabetizacao_BR.csv", "CD_DIST", "NM_DIST")
city_df = get_education_df("../data/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/Agregados_por_Municipio_csv/Agregados_por_municipios_alfabetizacao_BR.csv", "CD_MUN", "NM_MUN")
district_df.to_csv("education_district.csv", index=False)
city_df.to_csv("education_city.csv", index=False)

C:\Users\rgasi\AppData\Local\Temp\ipykernel_40488\271949840.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['perc alfa 15 a 19'] =  df['V00748'].divide(df['V00644'].replace(0, np.nan)).fillna(0)
C:\Users\rgasi\AppData\Local\Temp\ipykernel_40488\271949840.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['perc alfa 20 a 24'] =  df['V00749'].divide(df['V00645'].replace(0, np.nan)).fillna(0)
C:\Users\rgasi\AppData\Local\Temp\ipykernel_40488\271949840.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is 